In [1204]:
import pandas as pd
import datetime as dt
import yfinance as yf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

tickers = ['AAPL', 'TSLA', 'NFLX', 'GM', 'AMZN', 'GOOG', 'TGT']
etf = ['SPY', 'IWM', 'DIA']

In [1205]:
data = yf.download(tickers + etf, period = '10y') ['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [1206]:
df['portfolio'] = df.mean(axis=1)
returns= df.pct_change()

In [1207]:
# Part 1: Create Table showing constituent (stocks) risk analysis in the equal - weight portfolio analysis as of the current date

In [1208]:
# Column 1 - Ticker
output = pd.DataFrame(index=tickers)

In [1209]:
# Column 2 - Portfolio Weight (equally weighted)
output['Portfolio Weight'] = 100 / len(output.index)

In [1210]:
# Column 3 - Annualized Volatility (using trailing 3-months)
output['Annual Volatility'] = data[-63:].std() * np.sqrt(4)

In [1211]:
# Column 4 - Beta against SPY (using trailing 12-months)
# Citation - https://www.learnpythonwithrune.org/calculate-the-market-sp-500-beta-with-python-for-any-stock/#:~:text=Step%203%3A%20Calculate%20the%20BETA
log_returns = np.log(data/data.shift())
cov = log_returns.cov()
var = log_returns['SPY'].var()

cov.loc[tickers, 'SPY']/var
output['Beta against SPY'] = cov.loc[tickers, 'SPY']/var

In [1212]:
# Column 5 - Beta against IWM (using trailing 12-months)
log_returns = np.log(data/data.shift())
cov = log_returns.cov()
var = log_returns['IWM'].var()

cov.loc[tickers, 'IWM']/var
output['Beta against IWM'] = cov.loc[tickers, 'IWM']/var

In [1213]:
# Column 6 - Beta against DIA (using trailing 12-months)
log_returns = np.log(data/data.shift())
cov = log_returns.cov()
var = log_returns['DIA'].var()

cov.loc[tickers, 'DIA']/var
output['Beta against DIA'] = cov.loc[tickers, 'DIA']/var

In [1214]:
# Column 7 & 8 - Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High)
drawdown_5roll= log_returns[-252:].rolling(5).max()- log_returns[-252:].rolling(5).min()
output['avg_drawdown']=drawdown_5roll.mean()
output['max_drawdown']= drawdown_5roll.max()

In [1215]:
# Column 7 & 8 - Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High)
def Max_drawdown(df, window=5, draw_type = 'max') :
    if draw_type =='avg' :
        Roll_Max = df.rolling(window,min_periods=1) .mean()
    else:
        Roll_Max = df.rolling(window,min_periods=1) .max()
        
    Weekly_Drawdown= df/Roll_Max -1
    Max_Weekly_Drawdown = Weekly_Drawdown.rolling(window, min_periods=1) .min()
    
    return Max_Weekly_Drawdown

avg_drawdown = (Max_drawdown(data[tickers], draw_type = 'avg') .resample('Y') .mean()) .to_numpy()
avg_drawdown

max_drawdown = (Max_drawdown(data[tickers]).resample('Y') .mean()) .to_numpy()
max_drawdown

array([[-0.02792275, -0.06089693, -0.04335152, -0.02500138, -0.02612266,
        -0.01848547, -0.0188149 ],
       [-0.02188713, -0.05149634, -0.04595543, -0.03132294, -0.04071954,
        -0.02689035, -0.02134421],
       [-0.03284724, -0.04714096, -0.04432324, -0.03009838, -0.02685457,
        -0.02721262, -0.02534116],
       [-0.02611068, -0.04985475, -0.04676737, -0.03169043, -0.03262046,
        -0.02370809, -0.02538165],
       [-0.01583818, -0.04039983, -0.0269166 , -0.0244989 , -0.01844529,
        -0.01655234, -0.03541304],
       [-0.03477141, -0.06460606, -0.05249235, -0.03896051, -0.0386825 ,
        -0.0336932 , -0.03314119],
       [-0.02379441, -0.05551409, -0.0404262 , -0.02763956, -0.02454873,
        -0.02517729, -0.02203225],
       [-0.0410093 , -0.08177434, -0.04581665, -0.06786914, -0.03903093,
        -0.03707439, -0.03403499],
       [-0.0276489 , -0.05641565, -0.0341281 , -0.04345139, -0.03021957,
        -0.02261339, -0.02541688],
       [-0.04815714, -0.0893

In [1216]:
# Column 9 - Total Return (using trailing 10-years)
output['10yr Return'] = data.pct_change(len(data) - 1)[-1:].T * 100

In [1217]:
# Column 10 - Annualized Total Returns (Using trailing 10-years)
output['Ann 10yr Return'] = output['10yr Return'] ** (1 / np.sqrt(5)) 

ValueError: Wrong number of items passed 10, placement implies 1

In [ ]:
output

In [ ]:
# Part 2: Create a table showing Portfolio Risk against the three EFTs

In [ ]:
# Calculating 'ewp'
data['ewp'] = data[tickers].mean(axis=1)
log_returns['ewp'] = log_returns[tickers].mean(axis=1)

In [ ]:
# Calculating log_returns
log_returns[etf + ['ewp']]

In [ ]:
# Column 1- ETF Ticker
output_2 = pd.DataFrame(index=etf + ['ewp'])

In [ ]:
# Column 2 & 3 - Correlation and Covariance of Portfolio against ETF 
output_2['Correlation'] = log_returns.corr()['ewp']
output_2['Cov_2'] = (log_returns * 100).cov()['ewp']

In [ ]:
# Column 4 - Tracking Errors (using trailing 10-years)
output_2['tracking_error'] = 0
for bench in etf:
    output_2.loc[bench, 'Tracking Error'] = (log_returns[bench] - log_returns['ewp']).std() *100

In [ ]:
# Column 5 - Sharpe Ratio (using current risk-free rate)
(returns - (0.02)) / returns.std()

In [ ]:
# Column 6 - Annualized Volatility (252 days) Spread (portfolio volatility - ETF volatility)
output_2['volatility spread'] = log_returns[etf + ['ewp']][-252:].std() ** (1/252)

In [ ]:
output_2

In [ ]:
# Part 3: Create a correlation matrix showing the correlations between the equal-weighted portfolio, 3 ETFs, and your 7 stocks

In [ ]:
corr_data = log_returns[etf + tickers + ['ewp']][1:].corr(method= 'pearson')

In [ ]:
corr_data

In [ ]:
import seaborn as sns
sns.heatmap(corr_data, cmap= "YlGnBu")

In [ ]:
# Citation
- Credit given to Professor John Droescher
- https://www.learnpythonwithrune.org/calculate-the-market-sp-500-beta-with-python-for-any-stock/#:~:text=Step%203%3A%20Calculate%20the%20BETA 
- Hilpisch, Y. (2015). Python for finance. O'Reilly Media. 